# Exploring project datasets

Sparkify data reside in S3. There is a directory of JSON logs and a directory of JSON songs metadata. In this notebook, we will be looking closer at these files.

<a id='s1'></a>
## Table of contents

* [Loading user-editable configurations](#s2)
  * [Setting aws session](#s2-1)
  * [Setting s3 client](#s2-2)
* [Song data](#s3)
  * [S3 list of objects](#s3-1)
  * [S3 list of objects - number of rows](#s3-2)
  * [S3 list of objects - first object content](#s3-3)
* [Log data](#s4)
  * [Log data JSON path](#s4-1)
  * [S3 list of objects](#s4-2)
  * [S3 list of objects - number of rows](#s4-3)
  * [S3 list of objects - first object content](#s4-4)

In [1]:
from IPython.display import display

import pandas as pd
import boto3
import configparser
import os
import json

<a id='s2'></a>
## Loading user-editable configurations
[Go to Table of contents](#s1)

In the root directory there is a configuration file named 'dwh.default.cfg' with the variables needed to run the project.
Before setting those variables, make a copy of it to 'dwh.cf' file, which will be used.

In [2]:
# setting filepath of the configuration file
filepath = os.path.join('..', 'dwh.cfg')

# loading configuration file to memory
config = configparser.ConfigParser()
config.read_file(open(filepath))

<a id='s2-1'></a>
### Setting aws session
[Go to Table of contents](#s1)

In [3]:
# getting aws access variables 
KEY = config.get('AWS', 'KEY')
SECRET = config.get('AWS', 'SECRET')
REGION = 'us-west-2'

# setting aws session
aws = boto3.session.Session(
    aws_access_key_id=KEY,
    aws_secret_access_key=SECRET,
    region_name=REGION
)

<a id='s2-2'></a>

### Setting s3 client
[Go to Table of contents](#s1)

In [4]:
s3_client = aws.client('s3')

# loading user configurations of buckets
SONG_DATA = config.get('S3', 'SONG_DATA')
LOG_DATA = config.get('S3', 'LOG_DATA')
LOG_JSONPATH = config.get('S3', 'LOG_JSONPATH')

<a id='s3'></a>
## Song data
[Go to Table of contents](#s1)

Description given by udacity:

> The first dataset is a subset of real data from the Million Song Dataset. 
> Each file is in JSON format and contains metadata about a song and the artist of that song. 
> The files are partitioned by the first three letters of each song's track ID. 
> For example, here are filepaths to two files in this dataset.

<a id='s3-1'></a>
### S3 list of objects
[Go to Table of contents](#s1)

In [8]:
songdata_bucket, songdata_prefix = SONG_DATA.split('/')[2:4]
resp = s3_client.list_objects_v2(Bucket=songdata_bucket, Prefix=songdata_prefix)

df_songdata_files = pd.DataFrame(resp['Contents'])
# removing directories by getting just objects with size greater than 0
df_songdata_files = df_songdata_files[df_songdata_files['Size'] > 0].reset_index()

df_songdata_files[['Key', 'Size', 'LastModified']].head(3)

,Key,Size,LastModified
0,song_data/A/A/A/TRAAAAK128F9318786.json,225,2019-04-17 03:20:41+00:00
1,song_data/A/A/A/TRAAAAV128F421A322.json,284,2019-04-17 03:20:41+00:00
2,song_data/A/A/A/TRAAABD128F429CF47.json,249,2019-04-17 03:20:41+00:00


In [10]:
df_songdata_files['Size'].sum()

248679

<a id='s3-2'></a>
### S3 list of objects - number of rows
[Go to Table of contents](#s1)

In [13]:
df_songdata_files.size

5994

<a id='s3-3'></a>
### S3 list of objects - first object content
[Go to Table of contents](#s1)

In [14]:
first_object = df_songdata_files.loc[1,'Key']
resp = s3_client.get_object(Bucket=songdata_bucket, Key=first_object)
data = resp.get('Body').read()

json.loads(data)

{'artist_id': 'AR73AIO1187B9AD57B',
 'artist_latitude': 37.77916,
 'artist_location': 'San Francisco, CA',
 'artist_longitude': -122.42005,
 'artist_name': 'Western Addiction',
 'duration': 118.07302,
 'num_songs': 1,
 'song_id': 'SOQPWCR12A6D4FB2A3',
 'title': 'A Poor Recipe For Civic Cohesion',
 'year': 2005}

<a id='s4'></a>
## Log data
[Go to Table of contents](#s1)

Description given by udacity:
> The second dataset consists of log files in JSON format generated by this [event simulator](https://github.com/Interana/eventsim) based on the songs in the dataset above.
> These simulate app activity logs from an imaginary music streaming app based on configuration settings.
> <br />The log files in the dataset you'll be working with are partitioned by year and month. 

<a id='s4-1'></a>
### Log data JSON path
[Go to Table of contents](#s1)

In [5]:
logjson_bucket, logjson_key = LOG_JSONPATH.split('/')[2:4]

resp = s3_client.get_object(Bucket=logjson_bucket, Key=logjson_key)
data = resp.get('Body').read()

json.loads(data)

{'jsonpaths': ["$['artist']",
  "$['auth']",
  "$['firstName']",
  "$['gender']",
  "$['itemInSession']",
  "$['lastName']",
  "$['length']",
  "$['level']",
  "$['location']",
  "$['method']",
  "$['page']",
  "$['registration']",
  "$['sessionId']",
  "$['song']",
  "$['status']",
  "$['ts']",
  "$['userAgent']",
  "$['userId']"]}

In [7]:
songs_jsonpaths = {
    'jsonpaths': [
        "$['artist_id']",
        "$['artist_latitude']",
        "$['artist_location']",
        "$['artist_longitude']",
        "$['artist_name']",
        "$['duration']",
        "$['num_songs']",
        "$['song_id']",
        "$['title']",
        "$['year']"
    ]
}

s3_client.put_object(
    Body=json.dumps(songs_jsonpaths).encode('UTF-8'),
    Bucket='jsonpaths-dj',
    Key='songs.json'
)

{'ResponseMetadata': {'RequestId': 'YFC9R0SZEE2T1KKZ',
  'HostId': 'rD4JNO9ESusGkh665QBmFqx6aIpAZYkqEjwjaiedq7rFc2yIFZsLUV57Y792clc/t6pYJhhc9Vc=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'rD4JNO9ESusGkh665QBmFqx6aIpAZYkqEjwjaiedq7rFc2yIFZsLUV57Y792clc/t6pYJhhc9Vc=',
   'x-amz-request-id': 'YFC9R0SZEE2T1KKZ',
   'date': 'Fri, 19 Nov 2021 00:43:54 GMT',
   'etag': '"13e1cc00878874d37d7091ac4333d1e7"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"13e1cc00878874d37d7091ac4333d1e7"'}

<a id='s4-2'></a>
### S3 list of objects
[Go to Table of contents](#s1)

In [5]:
logdata_bucket, logdata_prefix = LOG_DATA.split('/')[2:4]
resp = s3_client.list_objects_v2(Bucket=logdata_bucket, Prefix=logdata_prefix)

df_logdata_files = pd.DataFrame(resp['Contents'])
# removing directories by getting just objects with size greater than 0
df_logdata_files = df_logdata_files[df_logdata_files['Size'] > 0].reset_index()

df_logdata_files[['Key', 'Size', 'LastModified']].head(3)

,Key,Size,LastModified
0,log_data/2018/11/2018-11-01-events.json,7151,2019-04-17 12:03:13+00:00
1,log_data/2018/11/2018-11-02-events.json,83585,2019-04-17 12:03:13+00:00
2,log_data/2018/11/2018-11-03-events.json,54084,2019-04-17 12:03:13+00:00


<a id='s4-3'></a>
### S3 list of objects - number of rows
[Go to Table of contents](#s1)

In [6]:
df_logdata_files.size

180

<a id='s4-4'></a>
### S3 list of objects - first object content
[Go to Table of contents](#s1)

In [7]:
first_object = df_logdata_files.loc[0,'Key']
resp = s3_client.get_object(Bucket=logdata_bucket, Key=first_object)
data = resp.get('Body').read()

df_first_object = pd.read_json(data, lines=True)
first_record = data.splitlines()[0]

json.loads(first_record)

{'artist': None,
 'auth': 'Logged In',
 'firstName': 'Walter',
 'gender': 'M',
 'itemInSession': 0,
 'lastName': 'Frye',
 'length': None,
 'level': 'free',
 'location': 'San Francisco-Oakland-Hayward, CA',
 'method': 'GET',
 'page': 'Home',
 'registration': 1540919166796.0,
 'sessionId': 38,
 'song': None,
 'status': 200,
 'ts': 1541105830796,
 'userAgent': '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"',
 'userId': '39'}

In [11]:
df_first_object.head(3)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1540919166796,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1540344794796,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
